<a href="https://colab.research.google.com/github/myPar/NSU_Practice/blob/dev/baseline/RNNbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objects as go

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/'
file_name = 'data.txt'
file_path = Path(dir_name + file_name)
df = pd.read_csv(filepath_or_buffer=file_path, sep=' ', header=0)

df.head

<bound method NDFrame.head of              Date      Time  qo_lc[m3/d]  qw_lc[m3/d]  qg_lc[m3/d]  \
0     07-May-2014  10:13:26      109.280    -0.312890       3810.3   
1     07-May-2014  10:14:26      113.460    -0.437350       3774.0   
2     07-May-2014  10:15:26      131.630    -0.533550       3892.5   
3     07-May-2014  10:16:26      140.670     0.160910       4043.1   
4     07-May-2014  10:17:26      150.020    -1.145600       4134.6   
...           ...       ...          ...          ...          ...   
1264  08-May-2014  07:34:35       98.744     0.310070       3174.5   
1265  08-May-2014  07:35:35       72.045     0.162760       3134.1   
1266  08-May-2014  07:36:35       87.370    -0.125770       3340.4   
1267  08-May-2014  07:37:35       32.121     0.021522       2834.1   
1268  08-May-2014  07:38:35       35.914     0.228020       2612.4   

      qo_sc[Sm3/d]  qw_sc[Sm3/d]  qg_sc[Sm3/d]  qo_scnp[Sm3/d]  \
0          108.430     -0.314870       92700.0         108.430 

In [ ]:
print(df.columns)

Index(['Date', 'Time', 'qo_lc[m3/d]', 'qw_lc[m3/d]', 'qg_lc[m3/d]',
       'qo_sc[Sm3/d]', 'qw_sc[Sm3/d]', 'qg_sc[Sm3/d]', 'qo_scnp[Sm3/d]',
       'qw_scnp[Sm3/d]', 'qg_scnp[Sm3/d]', 'Fo_lc[%]', 'Fw_lc[%]', 'Fg_lc[%]',
       'WLR[%]', 'GVF[%]', 'GLR[m3/m3]', 'BSW[%]', 'GOR[Sm3/Sm3]',
       'GOR1[Sm3/Sm3]', 'mo_lc[kg/d]', 'mw_lc[kg/d]', 'mg_lc[kg/d]',
       'm_lc[kg/d]', 'mo_sc[kg/d]', 'mw_sc[kg/d]', 'mg_sc[kg/d]',
       'Mu_o_lc[cP]', 'Mu_l_lc[cP]', 'Do_lc[g/cm3]', 'Dw_lc[g/cm3]',
       'Dg_lc[g/cm3]', 'Dm_lc[g/cm3]', 'Dl_lc[g/cm3]', 'bo[Sm3/m3]',
       'bw[Sm3/m3]', 'bg[Sm3/m3]', 'Z', 'Rst[Sm3/Sm3]', 'Rwst[Sm3/Sm3]',
       'rgmp[Sm3/Sm3]', 'N32[cps]', 'N81[cps]', 'N356[cps]', 'NTotal[cps]',
       'DeadTime[s]', 'SampleTime[s]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]',
       'TAMB[DegC]', 'Dyn_DP[mbar]', 'OperatingPointLE[1/m]',
       'OperatingPointHE[1/m]', 'OilPointLE[1/m]', 'OilPointHE[1/m]',
       'WaterPointLE[1/m]', 'WaterPointHE[1/m]', 'GasPointLE[1/m]',
       'GasPoin

In [ ]:
selected_rows = ['DPV[mbar]', 'PL[bara]', 'qg_sc[Sm3/d]', 'qo_sc[Sm3/d]', 'TL[DegC]']
outputs = ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']
inputs = np.setdiff1d(selected_rows, outputs)

df = df[selected_rows]

print(df.head)
print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

<bound method NDFrame.head of       DPV[mbar]  PL[bara]  qg_sc[Sm3/d]  qo_sc[Sm3/d]  TL[DegC]
0       1448.70    21.481       92700.0       108.430   -2.5452
1       1462.10    21.473       91895.0       112.580   -2.5323
2       1592.30    21.672       95860.0       130.550   -2.3931
3       1703.30    21.926      100690.0       139.440   -2.0451
4       1821.70    22.196      104290.0       148.640   -1.8009
...         ...       ...           ...           ...       ...
1264    1147.10    23.997       87402.0        97.703   -3.3542
1265    1090.50    23.919       85594.0        71.288   -3.2890
1266    1279.10    24.477       93660.0        86.386   -3.2191
1267     976.62    23.808       76564.0        31.789   -3.2819
1268        NaN       NaN       69609.0        35.567       NaN

[1269 rows x 5 columns]>
inputs: ['DPV[mbar]' 'PL[bara]' 'TL[DegC]']
outputs: ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']


In [ ]:
# check categorical attributes
cardinality_hold = 0.15
expected_col_size = 1269

categorical_attributes = []

def check_attributes_info():
  for column in df.columns:
    col_data = df[column].to_numpy()
    size = len(col_data)
    
    assert size == expected_col_size
    cleared_data = col_data[~np.isnan(col_data)]
    
    card = len(np.unique(cleared_data))  # cardinality without Nan items
    skip_count = size - len(cleared_data)

    print(column + ": card=" + str(card) + "; skip: percent=" + str(skip_count * 100 / size) + ", count=" + str(skip_count))
    
    if card / size < cardinality_hold:
      categorical_attributes.append(card)

check_attributes_info()

if len(categorical_attributes) == 0:
  print("no categorical attributes")
else:
  print(categorical_attributes)

DPV[mbar]: card=1163; skip: percent=0.07880220646178093, count=1
PL[bara]: card=1042; skip: percent=0.07880220646178093, count=1
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
TL[DegC]: card=1214; skip: percent=0.07880220646178093, count=1
no categorical attributes


In [ ]:
# supplement data
for column in df.columns:
  data =  np.array(df[column].to_numpy(), dtype=np.float64)

  for i in range(len(data)):
    if str(data[i]) == 'nan':
      data[i] = np.mean(data[~np.isnan(data)])
  df.drop([column], axis=1, inplace=True)
  df.insert(0, column, data)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
check_attributes_info()

TL[DegC]: card=1215; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
PL[bara]: card=1043; skip: percent=0.0, count=0
DPV[mbar]: card=1164; skip: percent=0.0, count=0


## Create model

In [ ]:
# model parameters
batch_size = 60
timesteps = 60
input_size = len(inputs)
output_size = len(outputs)
lr = 0.001

mse = tf.keras.losses.MeanSquaredError()

model = tf.keras.Sequential()
model.add(Input((None, input_size)))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=2, activation='tanh', return_sequences=True))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 120)         14880     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 120)         28920     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 2)           246       
                                                                 
Total params: 44,046
Trainable params: 44,046
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=mse, metrics=[mse], optimizer=Adam(learning_rate=lr))

## Prepare data

In [ ]:
# normilize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function

result_interval = interval_th
normalized_df = df = ((df - df.min()) / (df.max() - df.min())) * (result_interval[1] - result_interval[0]) + result_interval[0]

train_df, test_df = train_test_split(normalized_df, test_size=0.3)
print(train_df.head)
print(test_df.head)

train_input_data = train_df[inputs].to_numpy()
train_output_data = train_df[outputs].to_numpy()

test_input_data = test_df[inputs].to_numpy()
test_output_data = test_df[outputs].to_numpy()

print("data shapes:")
print(train_input_data.shape)
print(test_input_data.shape)

print(train_output_data.shape)
print(test_output_data.shape)

<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
1130 -0.657944     -0.596627     -0.312048  0.291349  -0.666149
442   0.893749      0.376286      0.556419 -0.131837   0.552215
409   0.334892     -0.125101     -0.127410 -0.576814  -0.192527
755   0.075409     -0.546891     -0.225494 -0.019830  -0.528073
880  -0.054370     -0.747312     -0.440479  0.039878  -0.647812
...        ...           ...           ...       ...        ...
435   0.975492     -0.154651      0.438197 -0.172805   0.338808
456   0.643013      0.021133      0.255640 -0.392024   0.144360
63   -0.240253     -0.722132     -0.691655 -0.630856  -0.746294
986  -0.177287     -0.342713      0.092620  0.476574  -0.362481
954  -0.152176     -0.297914      0.085133  0.345391  -0.277588

[888 rows x 5 columns]>
<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
249   0.613151     -0.520151     -0.083612 -0.661364  -0.210311
1254 -0.547093     

In [ ]:
# formatting the data
def get_sequence(data):
  return np.array([data[i*timesteps : i*timesteps + timesteps] for i in range(data.shape[0] // timesteps)])

def get_remaining_data(data, sequences_count):
  return np.array(data[timesteps * sequences_count:])

def print_sequences_info(in_seq, out_seq, in_rem, out_rem):
  print(in_seq.shape)
  print(out_seq.shape)

  for i in range(len(in_seq)):
    print(str(i) + ": " + str(in_seq[i].shape))

  print("-----------")

  for i in range(len(out_seq)):
    print(str(i) + ": " + str(out_seq[i].shape))

  print("remaining data: ")
  print(in_rem.shape)
  print(out_rem.shape)

input_train_sequences = get_sequence(train_input_data)
output_train_sequences = get_sequence(train_output_data)

input_test_sequences = get_sequence(test_input_data)
output_test_sequences = get_sequence(test_output_data)

input_train_remaining = []
output_train_remaining = []

input_test_remaining = []
output_test_remaining = []

train_sequences_count = len(input_train_sequences)
assert train_sequences_count == len(output_train_sequences)

test_sequences_count = len(input_test_sequences)
assert test_sequences_count == len(output_test_sequences)

# get the remaining data from train samples:
if train_sequences_count * timesteps < train_input_data.shape[0]:
  input_train_remaining = get_remaining_data(train_input_data, train_sequences_count)

if train_sequences_count * timesteps < train_output_data.shape[0]:
  output_train_remaining = get_remaining_data(train_output_data, train_sequences_count)

# get the remaining data from test samples:
if test_sequences_count * timesteps < test_input_data.shape[0]:
  input_test_remaining = get_remaining_data(test_input_data, test_sequences_count)

if test_sequences_count * timesteps < test_output_data.shape[0]:
  output_test_remaining = get_remaining_data(test_output_data, test_sequences_count)

print("train samples:")
print_sequences_info(input_train_sequences, output_train_sequences, input_train_remaining, output_train_remaining)
print("\n************************\n")
print("test samples:")
print_sequences_info(input_test_sequences, output_test_sequences, input_test_remaining, output_test_remaining)

train samples:
(14, 60, 3)
(14, 60, 2)
0: (60, 3)
1: (60, 3)
2: (60, 3)
3: (60, 3)
4: (60, 3)
5: (60, 3)
6: (60, 3)
7: (60, 3)
8: (60, 3)
9: (60, 3)
10: (60, 3)
11: (60, 3)
12: (60, 3)
13: (60, 3)
-----------
0: (60, 2)
1: (60, 2)
2: (60, 2)
3: (60, 2)
4: (60, 2)
5: (60, 2)
6: (60, 2)
7: (60, 2)
8: (60, 2)
9: (60, 2)
10: (60, 2)
11: (60, 2)
12: (60, 2)
13: (60, 2)
remaining data: 
(48, 3)
(48, 2)

************************

test samples:
(6, 60, 3)
(6, 60, 2)
0: (60, 3)
1: (60, 3)
2: (60, 3)
3: (60, 3)
4: (60, 3)
5: (60, 3)
-----------
0: (60, 2)
1: (60, 2)
2: (60, 2)
3: (60, 2)
4: (60, 2)
5: (60, 2)
remaining data: 
(21, 3)
(21, 2)


In [ ]:
def add_remaining(data, remaining):
  data = list(data)
  data.append(remaining)
  return data

input_train_tensor = add_remaining(input_train_sequences, input_train_remaining)
output_train_tensor = add_remaining(output_train_sequences, output_train_remaining)

input_test_tensor = add_remaining(input_test_sequences, input_test_remaining)
output_test_tensor = add_remaining(output_test_sequences, output_test_remaining)

print(len(input_train_tensor))
print(len(output_train_tensor))

def print_batches_shapes(in_tensor, out_tensor):
  print(" input batches shapes:")

  for i in range(len(in_tensor)):
    print(" " + str(in_tensor[i].shape))

  print(" ------------")
  print(" output batches shapes:")

  for i in range(len(out_tensor)):
    print(" " + str(out_tensor[i].shape))

print("train tensor:")
print_batches_shapes(input_train_tensor, output_train_tensor)
print()
print("test tensor:")
print_batches_shapes(input_test_tensor, output_test_tensor)

15
15
train tensor:
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (48, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (48, 2)

test tensor:
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (21, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (21, 2)


## Train model

In [ ]:
epoch_count = 500
batch_count = len(input_train_tensor)

input_shape = (1, timesteps, input_size)
output_shape = (1, timesteps, output_size)

input_shape_remaining = (1, input_train_remaining.shape[0], input_train_remaining.shape[1])
output_shape_remaining = (1, output_train_remaining.shape[0], output_train_remaining.shape[1])

#print("remainig shapes: " + str(input_shape_remaining) + ", " + str(output_shape_remaining))

loss_list = []

for epoch in range(epoch_count):
  for batch_idx in range(batch_count):
    in_shape = input_shape
    out_shape = output_shape

    if batch_idx == batch_count - 1:
      in_shape = input_shape_remaining 
      out_shape = output_shape_remaining

    input_batch = input_train_tensor[batch_idx].reshape(in_shape)
    output_batch = output_train_tensor[batch_idx].reshape(out_shape)

    loss = model.train_on_batch(input_batch, output_batch, reset_metrics=False, return_dict=True)
  loss_list.append(loss['loss'])

  print("epoch-" + str(epoch) + ";" + str(loss))

epoch-0;{'loss': 0.3613780438899994, 'mean_squared_error': 0.3613780438899994}
epoch-1;{'loss': 0.25893837213516235, 'mean_squared_error': 0.25893837213516235}
epoch-2;{'loss': 0.20289407670497894, 'mean_squared_error': 0.20289407670497894}
epoch-3;{'loss': 0.17756041884422302, 'mean_squared_error': 0.17756041884422302}
epoch-4;{'loss': 0.1535949409008026, 'mean_squared_error': 0.1535949409008026}
epoch-5;{'loss': 0.13494174182415009, 'mean_squared_error': 0.13494174182415009}
epoch-6;{'loss': 0.12065692245960236, 'mean_squared_error': 0.12065692245960236}
epoch-7;{'loss': 0.1100267544388771, 'mean_squared_error': 0.1100267544388771}
epoch-8;{'loss': 0.10202524065971375, 'mean_squared_error': 0.10202524065971375}
epoch-9;{'loss': 0.09555543959140778, 'mean_squared_error': 0.09555543959140778}
epoch-10;{'loss': 0.09059108793735504, 'mean_squared_error': 0.09059108793735504}
epoch-11;{'loss': 0.08596008270978928, 'mean_squared_error': 0.08596008270978928}
epoch-12;{'loss': 0.081363923847

In [ ]:
# plot loss graphic
x = np.arange(1, epoch_count + 1)  
y = np.array(loss_list)

assert len(x) == len(y)

fig = px.line(x=x, y=y, labels={'x':'epoch', 'y':'loss'})
fig.show()

## Test model

In [ ]:
# test on batches
batch_count = len(input_test_tensor)

input_shape = (1, timesteps, input_size)
output_shape = (1, timesteps, output_size)

input_shape_remaining = (1, input_test_remaining.shape[0], input_test_remaining.shape[1])
output_shape_remaining = (1, output_test_remaining.shape[0], output_test_remaining.shape[1])

loss = 0

for batch_idx in range(batch_count):
  in_shape = input_shape
  out_shape = output_shape

  if batch_idx == batch_count - 1:
    in_shape = input_shape_remaining 
    out_shape = output_shape_remaining

  input_batch = input_test_tensor[batch_idx].reshape(in_shape)
  output_batch = output_test_tensor[batch_idx].reshape(out_shape)

  loss = model.test_on_batch(input_batch, output_batch, return_dict=True, reset_metrics=False)

print("test loss=" + str(loss))

test loss={'loss': 0.006959015969187021, 'mean_squared_error': 0.006959015969187021}


## Making predictions

In [ ]:
# get sequences and remainig data:
input_data = df[inputs].to_numpy()
output_data = df[outputs].to_numpy()

print("data shapes:")
print(input_data.shape)
print(output_data.shape)

input_sequences = get_sequence(input_data)
output_sequences = get_sequence(output_data)

seq_count = len(input_sequences)
assert seq_count == len(output_sequences)

print("sequence count=" + str(seq_count))

input_remaining = get_remaining_data(input_data, seq_count)
output_remaining = get_remaining_data(output_data, seq_count)

# create tensors:
if len(input_remaining) > 0:
  has_remaining = True
  input_tensor = add_remaining(input_sequences, input_remaining)
  output_tensor = add_remaining(output_sequences, output_remaining)
else:
  has_remaining = False
  input_tensor = input_sequences
  output_tensor = output_sequences

batch_count = len(input_tensor)

input_shape = (1, timesteps, input_size)
input_shape_remaining = (1, input_remaining.shape[0], input_remaining.shape[1])

predictions = []

# make predictions:
for i in range(batch_count):
  in_shape = input_shape

  if i == batch_count - 1 and has_remaining:
    in_shape = input_shape_remaining 
  
  input_batch = input_tensor[i].reshape(in_shape)

  prediction = model.predict_on_batch(input_batch)
  predictions.append(prediction)
  print(prediction.shape)

data shapes:
(1269, 3)
(1269, 2)
sequence count=21
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 60, 2)
(1, 9, 2)


In [ ]:
# plot approximation graphic
def get_1d_data(tensor):
  size = len(tensor)
  result = tensor[0][0]

  for i in range(size - 1):
    item = tensor[i + 1][0]
    result = np.concatenate((result, item), axis=0)

  return result

def get_attribute_data(attribute_idx, one_dim_data):
  result = []
  
  for item in one_dim_data:
    result.append(item[attribute_idx])
  
  return np.array(result)


def plot_approximation(attribute_name, predicted_data, mode):
  comparison_graphic = go.Figure()

  x = np.arange(1, input_data.shape[0])
  y_actual = df[attribute_name].to_numpy()
  y_predicted = predicted_data

  comparison_graphic.add_trace(go.Scatter(x=x, y=y_actual,
                    mode=mode,
                    name=attribute_name + ' actual graphic',))

  comparison_graphic.add_trace(go.Scatter(x=x, y=y_predicted,
                    mode=mode,
                    name=attribute_name + ' predicted',))

  comparison_graphic.update_layout(
    height=900,
    width = 4000,
    title_text=attribute_name
  )
  return comparison_graphic

one_dim_predicted_data = get_1d_data(predictions)

print(one_dim_predicted_data.shape)

predicted_data1 = get_attribute_data(0, one_dim_predicted_data)
predicted_data2 = get_attribute_data(1, one_dim_predicted_data)

print("predicted data shapes: " + str(predicted_data1.shape) + " " + str(predicted_data2.shape))

(1269, 2)
predicted data shapes: (1269,) (1269,)


In [ ]:
plot_approximation(outputs[0], predicted_data1, 'markers').show()

In [ ]:
plot_approximation(outputs[1], predicted_data2, 'markers').show()

In [ ]:
# plot as graphics
plot_approximation(outputs[0], predicted_data1, 'lines').show()

In [ ]:
plot_approximation(outputs[1], predicted_data2, 'lines').show()

In [ ]:
# calc R^2 for predected attributes
def get_determination(actual_data, predicted_data):
  data_size = len(actual_data)
  assert data_size == len(predicted_data)

  actual_mean = np.mean(actual_data)
  actual_mean_vector = np.full((data_size,), actual_mean)

  arg1 = np.linalg.norm((actual_data - predicted_data).reshape(1,data_size), ord='fro')**2

  arg2 = np.linalg.norm((actual_mean_vector - actual_data).reshape(1, data_size), ord='fro')**2

  return 1 - arg1 / arg2

actual_data1 = df[outputs[0]].to_numpy()
actual_data2 = df[outputs[1]].to_numpy()

k_1 = get_determination(predicted_data1, actual_data1)
k_2 = get_determination(predicted_data2, actual_data2)

print(outputs[0] + " determination=" + str(k_1))
print(outputs[1] + " determination=" + str(k_2))

qg_sc[Sm3/d] determination=0.9079017184898847
qo_sc[Sm3/d] determination=0.33401418262968297


## Save model

In [ ]:
# save model in the default TensorFlow SavedModel format
path = Path('/content/drive/MyDrive/Colab Notebooks/RNN_model')
model.save(path)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/RNN_model/assets


## Elman net baseline

### prepare data

In [7]:
file_name = 'DataVX.xlsx'
file_path = Path(dir_name + file_name)
print(file_path)

/content/drive/MyDrive/Colab Notebooks/DataVX.xlsx


In [11]:
df1 = pd.read_excel(file_path, sheet_name=0)
df1.head()

,Date & Time,qo_lc[m3/d],qw_lc[m3/d],qg_lc[m3/d],qo_sc[Sm3/d],qw_sc[Sm3/d],qg_sc[Sm3/d],qo_scnp[Sm3/d],qw_scnp[Sm3/d],qg_scnp[Sm3/d],...,OperatingPointHE[1/m],OilPointLE[1/m],OilPointHE[1/m],WaterPointLE[1/m],WaterPointHE[1/m],GasPointLE[1/m],GasPointHE[1/m],co1min_qo_lc[m3/d],co1min_qw_lc[m3/d],co1min_qg_lc[m3/d]
0,07-May-2014 10:13:26,109.28,-0.31289,3810.3,108.43,-0.31487,92700,108.43,-0.31487,91298,...,1.00060,23.568,14.781,171.01,25.097,0.49280,0.32761,222.74,NaN,NaN
1,07-May-2014 10:14:26,113.46,-0.43735,3774.0,112.58,-0.44011,91895,112.58,-0.44011,90438,...,0.98577,23.570,14.782,171.01,25.097,0.49042,0.32603,354.37,NaN,NaN
2,07-May-2014 10:15:26,131.63,-0.53355,3892.5,130.55,-0.53691,95860,130.55,-0.53691,94153,...,1.04660,23.568,14.781,171.01,25.097,0.49134,0.32665,272.30,NaN,NaN
3,07-May-2014 10:16:26,140.67,0.16091,4043.1,139.44,0.16191,100690,139.44,0.16191,98844,...,1.11880,23.562,14.777,171.00,25.096,0.49673,0.33023,422.32,NaN,NaN
4,07-May-2014 10:17:26,150.02,-1.14560,4134.6,148.64,-1.15270,104290,148.64,-1.15270,102300,...,1.14540,23.553,14.771,170.99,25.094,0.50210,0.33380,289.40,NaN,NaN


In [12]:
df2 = pd.read_excel(file_path, sheet_name=1)
df2.head()

,qo_lc[m3/d],qw_lc[m3/d],qg_lc[m3/d],Date,co1min_qo_lc[m3/d],co1min_qw_lc[m3/d],co1min_qg_lc[m3/d],qo_lc_avg10,qo_lc_avg5,qo_lc_avg20,...,qw_lc_avg10,qw_lc_avg5,qw_lc_avg20,qw_lc_avg30,qw_lc_avg60,qg_lc_avg10,qg_lc_avg5,qg_lc_avg20,qg_lc_avg30,qg_lc_avg60
0,109.28,-0.31289,3810.3,07-May-2014 10:13:26,222.74,-0.75024,7584.3,109.7028,129.0120,103.18940,...,-0.368397,-0.453696,-0.342605,-0.350181,-0.374134,3868.08,3930.90,3775.645,3688.586667,3481.415000
1,113.46,-0.43735,3774.0,07-May-2014 10:14:26,354.37,-1.28379,11476.8,96.6760,90.3936,84.34390,...,-0.316812,-0.283099,-0.634228,-0.398086,2.503358,3683.21,3805.26,3379.860,3274.243333,3364.313333
2,131.63,-0.53355,3892.5,07-May-2014 10:15:26,495.04,-1.12288,15519.9,81.4646,103.3322,83.16180,...,-0.365334,-0.456182,-0.145568,3.357279,0.561701,3514.47,3820.22,3288.740,3295.953333,3465.146667
3,140.67,0.16091,4043.1,07-May-2014 10:16:26,645.06,-2.26848,19654.5,87.2232,90.0198,86.27455,...,-0.903123,-0.177442,0.162339,1.649437,0.611184,3245.25,3546.20,3247.110,3432.673333,3452.058333
4,150.02,-1.14560,4134.6,07-May-2014 10:17:26,784.44,-3.10951,23994.7,73.3470,69.6328,94.84145,...,-0.312707,-0.540187,7.118775,0.338276,0.184388,3153.03,3489.38,3419.875,3432.516667,3475.510000


In [13]:
selected_rows = ['co1min_qo_lc[m3/d]', 'co1min_qg_lc[m3/d]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]']
outputs = ['co1min_qo_lc[m3/d]', 'co1min_qg_lc[m3/d]']
inputs = np.setdiff1d(selected_rows, outputs)

print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

inputs: ['DPV[mbar]' 'PL[bara]' 'TL[DegC]']
outputs: ['co1min_qo_lc[m3/d]', 'co1min_qg_lc[m3/d]']


In [19]:
sheet1_columns = df1.columns
sheet2_columns = df2.columns

def print_columns(columns):
  for i in range(len(columns)):
    col = sheet1_columns[i]
    if (i + 1) % 6 == 0:
      print(str(col) + " ")
    else:
      print(str(col) + " ", end='')

print("sheet1 columns: ")
print_columns(sheet1_columns)
print('\n\n')
print("sheet2 columns: ")
print_columns(sheet2_columns)

sheet1 columns: 
Date & Time                   qo_lc[m3/d]          qw_lc[m3/d]          qg_lc[m3/d]         qo_sc[Sm3/d]         qw_sc[Sm3/d] 
        qg_sc[Sm3/d]       qo_scnp[Sm3/d]       qw_scnp[Sm3/d]       qg_scnp[Sm3/d]             Fo_lc[%]             Fw_lc[%] 
            Fg_lc[%]               WLR[%]               GVF[%]           GLR[m3/m3]               BSW[%]         GOR[Sm3/Sm3] 
       GOR1[Sm3/Sm3]          mo_lc[kg/d]          mw_lc[kg/d]          mg_lc[kg/d]           m_lc[kg/d]          mo_sc[kg/d] 
         mw_sc[kg/d]          mg_sc[kg/d]          Mu_o_lc[cP]          Mu_l_lc[cP]         Do_lc[g/cm3]         Dw_lc[g/cm3] 
        Dg_lc[g/cm3]         Dm_lc[g/cm3]         Dl_lc[g/cm3]           bo[Sm3/m3]           bw[Sm3/m3]           bg[Sm3/m3] 
                   Z         Rst[Sm3/Sm3]        Rwst[Sm3/Sm3]        rgmp[Sm3/Sm3]             N32[cps]             N81[cps] 
           N356[cps]          NTotal[cps]          DeadTime[s]        SampleTime[s]           